In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

In [2]:
## preprocessing of data

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])

holiday_event_data_path = '/Users/sandeep/Desktop/store-sales-time-series-forecasting/holidays_events.csv'
oil_data_path = '/Users/sandeep/Desktop/store-sales-time-series-forecasting/oil.csv'
stores_data_path = '/Users/sandeep/Desktop/store-sales-time-series-forecasting/stores.csv'
test_data_path = '/Users/sandeep/Desktop/store-sales-time-series-forecasting/test.csv'
train_data_path = '/Users/sandeep/Desktop/store-sales-time-series-forecasting/train.csv'
transactions_data_path = '/Users/sandeep/Desktop/store-sales-time-series-forecasting/transactions.csv'

df_oil=pd.read_csv(oil_data_path)
df_oil = df_oil.infer_objects(copy=False)
df_oil.interpolate(method='linear',inplace=True)
df_oil.bfill(inplace=True)

df_train=pd.read_csv(train_data_path)
df_test=pd.read_csv(test_data_path)
df_stores=pd.read_csv(stores_data_path)
df_holiday_event=pd.read_csv(holiday_event_data_path)
df_transactions=pd.read_csv(transactions_data_path)
# print(df_oil.head(50))

/var/folders/d3/sh_q5_g16hz6flw_fh4nr7nh0000gn/T/ipykernel_48952/2414586291.py:14: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_oil.interpolate(method='linear',inplace=True)


In [3]:
## now merge all data into one file with specific features that are helpfull for prediction

train_data=pd.merge(df_train,df_stores,on='store_nbr',how='left') ## merge df_train and df_test with df_stores with respect to commen store_nbr column
test_data=pd.merge(df_test,df_stores,on='store_nbr',how='left')

train_data=pd.merge(train_data,df_oil,on='date',how='left') ## merge train_data and test_data with df_oil with respect to date column
test_data=pd.merge(test_data,df_oil,on='date',how='left')

train_data=pd.merge(train_data,df_holiday_event,on='date',how='left') ## merge train_data and test_data with df_holiday_event with respect to date column
test_data=pd.merge(test_data,df_holiday_event,on='date',how='left')

train_data=pd.merge(train_data,df_transactions,on=['date','store_nbr'],how='left')
test_data=pd.merge(test_data,df_transactions,on=['date','store_nbr'],how='left')


In [4]:
# print(train_data.head())
## now han

   id        date  store_nbr      family  sales  onpromotion   city  \
0   0  2013-01-01          1  AUTOMOTIVE    0.0            0  Quito   
1   1  2013-01-01          1   BABY CARE    0.0            0  Quito   
2   2  2013-01-01          1      BEAUTY    0.0            0  Quito   
3   3  2013-01-01          1   BEVERAGES    0.0            0  Quito   
4   4  2013-01-01          1       BOOKS    0.0            0  Quito   

       state type_x  cluster  dcoilwtico   type_y    locale locale_name  \
0  Pichincha      D       13       93.14  Holiday  National     Ecuador   
1  Pichincha      D       13       93.14  Holiday  National     Ecuador   
2  Pichincha      D       13       93.14  Holiday  National     Ecuador   
3  Pichincha      D       13       93.14  Holiday  National     Ecuador   
4  Pichincha      D       13       93.14  Holiday  National     Ecuador   

          description transferred  transactions  
0  Primer dia del ano       False           NaN  
1  Primer dia del ano 